# MoodCoin 모델링. 뻐킹 GSM 페스티벌

In [1]:
from google.colab import drive
# !pip install numpy==1.16.2
import numpy as np

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

class Data():
    def __init__(self, maxlen=20):
        # np_load_old = np.load
        # # modify the default parameters of np.load
        # np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
        # # restore np.load for future normal usage
        # np.load = np_load_old
        X = np.load('/content/gdrive/My Drive/project/MoodCoin/X_data.npy')
        Y = np.load('/content/gdrive/My Drive/project/MoodCoin/Y_data.npy')

        X = pad_sequences(X, maxlen=maxlen)
        Y = to_categorical(Y)

        s = np.arange(X.shape[0])
        np.random.shuffle(s)
        X = X[s]
        Y = Y[s]

        X_train = X[:-500]
        Y_train = Y[:-500]
        X_test = X[-500:]
        Y_test = Y[-500:]

        self.shape = X_train.shape[1:]
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
        self.maxlen = maxlen
        self.class_number = len(self.Y_test[0])

Using TensorFlow backend.


In [0]:
from keras import layers
from keras.models import Sequential

def build_model(input_size, max_features, output_size):
    m = Sequential()
    m.add(layers.Embedding(max_features, 32, input_shape=input_size))
    m.add(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    # m.add(layers.Dropout(0.2))
    
    m.add(layers.Dense(16, activation='relu'))
    m.add(layers.Dense(output_size, activation='softmax'))

    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return m

In [0]:
import matplotlib.pyplot as plt

def plot_graph(history, yname, title=None):
    # summarize history for accuracy
    if not isinstance(history, dict):
        history = history.history

    if yname == 'acc':
        plt.plot(history['acc']) # 선 그리기
        plt.plot(history['val_acc'])
        plt.ylabel('Accuracy') # y축 이름 표시
    else:
        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        plt.ylabel('Loss') # y축 이름 표시
        
    if title is not None:
        plt.title(title) # 그래프 제목 표시
    
    plt.xlabel('Epoch') # x축 이름 표시
    plt.legend(['Training', 'Verification'], loc=0) # 각 라인의 표식 표시
    plt.show()

In [5]:
data = Data()
print(data.X_train[0].shape)
model = build_model(data.shape, data.maxlen, data.class_number)
model.summary()

ValueError: ignored

In [0]:
history = model.fit(data.X_train, data.Y_train, batch_size=5, epochs=10, validation_split=0.2)

In [0]:
score = model.evaluate(data.X_test, data.Y_test)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

In [0]:
plot_graph(history, 'acc')
plot_graph(history, 'loss')

In [0]:
model.save('/content/gdrive/My Drive/project/MoodCoin/model.h5')

In [0]:
# d = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4], [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
#          0,   0,   0,   0, 706,  34, 151], [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
#          0,   0,   0,   0,   0,   2, 151], [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
#          0,   0,   0,   0, 354, 701,   2]])

emotion = ['감정 없음', '기쁨', '분노', '불안', '슬픔', '짜증']
s = np.arange(data.X_train.shape[0])
np.random.shuffle(s)
print(s[:10])
d = data.X_train[s[:10]]
y = data.Y_train[s[:10]]

print([emotion[np.argmax(pred)] for pred in model.predict(d)])
print([np.argmax(Y) for Y in y])